# word2vec 단어 간 유사도 체크
- 최대한 육아를 주제로 한 말뭉치를 모아보았다. (Total 2,755 documents)
- 우리가 원하는 의도에 일치하는 단어 간 유사도 결과가 나오는지를 확인하는 것이 목적이다.

## Import

In [1]:
import pandas as pd
import numpy as np
import re

from konlpy.tag import Mecab
from konlpy.tag import Komoran
from gensim.models import Word2Vec

### Datas

In [2]:
df1 = pd.read_csv("data/maeili.csv", encoding='utf-8')
df2 = pd.read_csv("data/baby.csv", encoding='utf-8')
df3 = pd.read_csv("data/toys.csv", encoding='utf-8')
df4 = pd.read_csv("data/seoulchildcare.csv", encoding='utf-8')
df5 = pd.read_csv("data/babynews_DLC.csv", encoding='utf-8')
df6 = pd.read_csv("data/ildongfoodis.csv", encoding='utf-8')
df7 = pd.read_csv("data/momsmagazine.csv", encoding='utf-8')
df8 = pd.read_csv("data/ange.csv", encoding='utf-8')
df9 = pd.read_csv("data/babytimes.csv", encoding='utf-8')
df10 = pd.read_csv("data/daekyo.csv", encoding='utf-8')
df11 = pd.read_csv("data/mom&enfant.csv", encoding='utf-8')
df12 = pd.read_csv("data/namyang.csv", encoding='utf-8')

### Concatenate Dataframe

In [3]:
df = pd.concat([df1.text, df2.text, df3.text, df4.text, df5.text,
                df6.text, df7.text, df8.text, df9.text, df10.text,
                df11.text, df12.text])
df = pd.DataFrame(df).reset_index(drop=True)
df.tail()

,text
26956,임신을 축하드립니다 임신을 확인한 후에 느끼는 기쁨과 설레임 이제는 혼자가 아니기에...
26957,임신을 한 순간부터 태교는 시작된다 태교는 단순히 영재를 출산하기 위해서가 아니라 ...
26958,16 20주 6개월 태아는 엄마 배꼽 아래에 있으며 자궁은 태아가 자유롭게 움직일 ...
26959,현대의학의 힘으로는 풀아하지 못하는 영역 중의 하나가 태몽이다 신기하게 임신만 하면...
26960,속설1 배가 부른 모양을 보면 아들 딸 여부를 알 수 있다 주부 설문 그렇다 84 ...


### Preprocessing

In [4]:
def cleaningText(text):
    text = re.sub('&nbsp;|\xa0|\u200b|\r|\n|\t', ' ', text)
    text = re.sub('<.+>|【.+】|\[.+\]', ' ', text)
    text = re.sub('[-_=+,/\?;:^$.@*\"※~&%ㆍ!”“’「」『』〈〉\\‘|\(\)\{\}\[\]\<\>`\'…《》]', ' ', text)
    text = re.sub('[㈜×–～→▪‧•∘✽○□◇△▽▷◁☆◈◐▶▼▲◆★♦■♥️●♠🖤]', ' ', text)
    text = re.sub('[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳]', ' ', text)
    text = re.sub('[➊❷➌❹❺❻❼❽❾❿]', ' ', text)
    text = re.sub('http.+com|http.+co|http.+kr|www.+kr|www.+com|www.+net|http.+net', ' ', text)
    text = re.sub("\s.+\s기자", '', text)
    text = re.sub('ⓒ\S+\s|㈜\S+\s|홈페이지\S+\s|전화\S+\s|대표전화\S+\s|☎\S+\s|http\S+\s', '', text)
    
    text = re.sub('\s+', ' ', text)
    text = text.strip()

    return text


In [5]:
df = df.drop(index=[952, 955, 956, 960, 962, 964, 1165, 1240, 1322])
df = df.drop_duplicates()
df = df.dropna()
df = pd.DataFrame(df).reset_index(drop=True)

In [6]:
df['text'] = df['text'].apply(cleaningText)
df = df.dropna()
df.to_csv('data/text_for_word2vec.csv', index=False, encoding='utf-8')

In [7]:
df

,text
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...
...,...
25290,임신을 축하드립니다 임신을 확인한 후에 느끼는 기쁨과 설레임 이제는 혼자가 아니기에...
25291,임신을 한 순간부터 태교는 시작된다 태교는 단순히 영재를 출산하기 위해서가 아니라 ...
25292,16 20주 6개월 태아는 엄마 배꼽 아래에 있으며 자궁은 태아가 자유롭게 움직일 ...
25293,현대의학의 힘으로는 풀아하지 못하는 영역 중의 하나가 태몽이다 신기하게 임신만 하면...


### Add tokenized column

191210 정체불명의 error 발생
```
NotImplementedError: Wrong number or type of arguments for overloaded function 'Tagger_parse'.
  Possible C/C++ prototypes are:
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)
    MeCab::Tagger::parse(MeCab::Lattice *) const
    MeCab::Tagger::parse(char const *)
```

babynews 데이터가 추가된 후 갑자기 발생한 에러인데.. 왜인지 모르겠지만 해당 부분 데이터프레임만 따로 tokenize하는 것은 문제없이 돌아간다.  
따라서 번거롭지만 분리하여 tokenizing 한 후에 합치기로 하였다.  

-> 해결.. `NaN` value 문제였다(어이없게도 NaN 값이 중간에 섞여있었음;). 누락 데이터를 잘 체크하는 습관을 갖도록하자.

In [8]:
def tokenize(text):
    tokenizer = Mecab()
    text = tokenizer.morphs(text)
#     text = ' '.join(text)
    return text

In [9]:
%%time
df['tokenized'] = df['text'].apply(tokenize)
df.tail()

CPU times: user 1min 4s, sys: 31.2 s, total: 1min 35s
Wall time: 1min 35s


,text,tokenized
25290,임신을 축하드립니다 임신을 확인한 후에 느끼는 기쁨과 설레임 이제는 혼자가 아니기에...,"[임신, 을, 축하, 드립니다, 임신, 을, 확인, 한, 후, 에, 느끼, 는, 기..."
25291,임신을 한 순간부터 태교는 시작된다 태교는 단순히 영재를 출산하기 위해서가 아니라 ...,"[임신, 을, 한, 순간, 부터, 태교, 는, 시작, 된다, 태교, 는, 단순히, ..."
25292,16 20주 6개월 태아는 엄마 배꼽 아래에 있으며 자궁은 태아가 자유롭게 움직일 ...,"[16, 20, 주, 6, 개월, 태아, 는, 엄마, 배꼽, 아래, 에, 있, 으며..."
25293,현대의학의 힘으로는 풀아하지 못하는 영역 중의 하나가 태몽이다 신기하게 임신만 하면...,"[현대, 의학, 의, 힘, 으로, 는, 풀, 아, 하, 지, 못하, 는, 영역, 중..."
25294,속설1 배가 부른 모양을 보면 아들 딸 여부를 알 수 있다 주부 설문 그렇다 84 ...,"[속설, 1, 배, 가, 부른, 모양, 을, 보, 면, 아들, 딸, 여부, 를, 알..."


In [10]:
df.isnull().sum()

text         0
tokenized    0
dtype: int64

## Train Word2Vec

#### WTF error
- df를 csv 형식으로 저장한 후 다시 로드하면 Word2Vec 학습에 인코딩 문제가 생긴다.
- 더 자세히 파고들어보니 list 형식의 column이 csv로 저장할 때 강제로 string 타입으로 형변환 됨.
- 이렇게 형변환 된 column을 Word2Vec 학습에 사용할 시 한글 텍스트가 깨지는 현상이 발생함.. 정확한 원인은 아직 모르겠다.

In [11]:
# df = pd.read_csv('text_for_word2vec.csv', encoding='utf-8')

그러므로 저장 & 로드 과정 생략하고 바로 말뭉치를 numpy ndarray형식으로 변환

In [12]:
corpus = np.array(df.tokenized)
type(corpus)

numpy.ndarray

In [13]:
%%time
model_all = Word2Vec(corpus, size=128, workers=4, sg=1)
model_all.save('data/tokmom_word2vec_all.model')

CPU times: user 6min 37s, sys: 655 ms, total: 6min 38s
Wall time: 1min 45s


#### 유사어 분석

In [14]:
model_all.wv.similar_by_word("놀이")

[('소꿉', 0.7411174774169922),
 ('공놀이', 0.71453857421875),
 ('도미노', 0.6938313245773315),
 ('기차놀이', 0.6872760057449341),
 ('지점토', 0.6789287328720093),
 ('음률', 0.6786611080169678),
 ('실뜨기', 0.6781467199325562),
 ('점토', 0.670332670211792),
 ('보물찾기', 0.6694741249084473),
 ('역할극', 0.6654926538467407)]

In [15]:
model_all.wv.similar_by_word("아이")

[('어린아이', 0.7223340272903442),
 ('아기', 0.7108864784240723),
 ('갓난아이', 0.6619225144386292),
 ('갓난아기', 0.6425191164016724),
 ('돌아와도', 0.6245362758636475),
 ('엄마', 0.6201267242431641),
 ('큰아이', 0.619462788105011),
 ('물고', 0.6189855933189392),
 ('친해진다', 0.6176779270172119),
 ('딸아이', 0.6112887263298035)]

In [16]:
model_all.wv.similar_by_word("치아")

[('잇몸', 0.8351531028747559),
 ('영구치', 0.78892982006073),
 ('충치', 0.7724026441574097),
 ('젖니', 0.7370181083679199),
 ('턱뼈', 0.7262221574783325),
 ('앞니', 0.7241079211235046),
 ('혀', 0.7098507881164551),
 ('어금니', 0.7040728330612183),
 ('에나멜질', 0.6952147483825684),
 ('법랑질', 0.6951456069946289)]

In [17]:
model_all.wv.similar_by_word("이빨")

[('혓바닥', 0.7814816236495972),
 ('거뭇', 0.7492300271987915),
 ('뜯어내', 0.7477034330368042),
 ('새빨간', 0.7451372146606445),
 ('허옇', 0.7419379353523254),
 ('댄다면', 0.7389610409736633),
 ('거스러미', 0.7372226715087891),
 ('영인', 0.736304759979248),
 ('싹싹', 0.7329772710800171),
 ('핥', 0.7311192750930786)]

In [18]:
model_all.wv.similar_by_word("감기")

[('목감기', 0.8299115896224976),
 ('코감기', 0.8279785513877869),
 ('기침감기', 0.7871049642562866),
 ('독감', 0.7644734382629395),
 ('냉방병', 0.7553010582923889),
 ('호흡기', 0.75002121925354),
 ('편도염', 0.7488691806793213),
 ('기관지염', 0.7488519549369812),
 ('인후염', 0.7480571269989014),
 ('잔병', 0.7451233863830566)]

In [19]:
model_all.wv.similar_by_word("열")

[('고열로', 0.6023843288421631),
 ('오한', 0.587587296962738),
 ('미열', 0.5840661525726318),
 ('고열', 0.5773072242736816),
 ('들어온다고', 0.5705861449241638),
 ('열기', 0.562213122844696),
 ('배탈', 0.5500274300575256),
 ('식혀', 0.5463265776634216),
 ('닫', 0.5445263385772705),
 ('사병', 0.5416484475135803)]

In [20]:
model_all.wv.similar_by_word("아파")

[('아프', 0.8031877279281616),
 ('아파서', 0.8024438619613647),
 ('다쳐', 0.7528544664382935),
 ('아픈데', 0.7512003183364868),
 ('아파도', 0.7441445589065552),
 ('지끈지끈', 0.7414534687995911),
 ('거린다고', 0.7205301523208618),
 ('주물렀', 0.7180495262145996),
 ('다쳐서', 0.7130210399627686),
 ('토해서', 0.7102508544921875)]

In [21]:
model_all.wv.similar_by_word("아프")

[('아파', 0.8031877875328064),
 ('아파서', 0.760517954826355),
 ('지끈지끈', 0.7557467222213745),
 ('결리', 0.7497872114181519),
 ('아파하', 0.7446613311767578),
 ('간지럽', 0.7426501512527466),
 ('쑤시', 0.7392691373825073),
 ('어지러우', 0.7379573583602905),
 ('거린', 0.7342636585235596),
 ('먹먹해', 0.7338392734527588)]

In [22]:
model_all.wv.similar_by_word("울음")

[('터뜨린', 0.7119895219802856),
 ('터뜨릴', 0.7058272361755371),
 ('터뜨립니다', 0.6808439493179321),
 ('자지러질', 0.6708759069442749),
 ('터뜨린다', 0.6708478927612305),
 ('옹알이', 0.6670913696289062),
 ('지르', 0.6653952598571777),
 ('큰소리', 0.6627084016799927),
 ('질러', 0.6608275771141052),
 ('고래고래', 0.6585405468940735)]

In [23]:
model_all.wv.similar_by_word("남아")

[('여아', 0.8500170707702637),
 ('정상아', 0.6760383248329163),
 ('정상인', 0.6114009618759155),
 ('여자', 0.6078804731369019),
 ('018', 0.6035465002059937),
 ('남자', 0.6023576259613037),
 ('초산부', 0.5972046852111816),
 ('클리토리스', 0.5966171026229858),
 ('3540', 0.5953346490859985),
 ('아든', 0.5947675704956055)]

In [24]:
model_all.wv.similar_by_word("여아")

[('남아', 0.8500170707702637),
 ('여자', 0.6377266049385071),
 ('남자', 0.6137702465057373),
 ('아든', 0.6029308438301086),
 ('617', 0.5642774701118469),
 ('018', 0.5627574920654297),
 ('1053', 0.5565885305404663),
 ('정상아', 0.5555915832519531),
 ('603', 0.5549695491790771),
 ('412', 0.5466080904006958)]

---

## Only Noun

- 이번에는 말뭉치에서 명사만 추출하여 결과를 비교해보자
- 유사한 키워드 잡기를 목적으로 할 때는 명사만 추출하는 것이 더 적절한 듯

In [25]:
%%time
def extract_noun(text):
    tokenizer = Mecab()
    text = tokenizer.nouns(text)
#     text = ' '.join(text)
    return text
    
df['noun'] = df['text'].apply(extract_noun)
df = df.dropna()

CPU times: user 1min 5s, sys: 30.7 s, total: 1min 36s
Wall time: 1min 35s


In [26]:
noun_corpus = np.array(df.noun)

In [48]:
%%time
model_noun = Word2Vec(noun_corpus, size=100, workers=4, sg=1)
model_noun.save('data/tokmom_word2vec_only_noun.model')

CPU times: user 2min 55s, sys: 210 ms, total: 2min 55s
Wall time: 47.7 s


In [49]:
model_noun.wv.similar_by_word("육아")

[('워킹맘', 0.6994822025299072),
 ('자처', 0.6550484299659729),
 ('독박', 0.6505540609359741),
 ('체험담', 0.6436377763748169),
 ('장세희', 0.6367778778076172),
 ('바이블', 0.6353555917739868),
 ('아서', 0.6345869898796082),
 ('살림', 0.6286143064498901),
 ('양육', 0.6285918354988098),
 ('휴직', 0.6241766214370728)]

In [50]:
model_noun.wv.similar_by_word("시댁")

[('친정', 0.8590857982635498),
 ('시부모', 0.8191248178482056),
 ('식구', 0.8130761384963989),
 ('본가', 0.8126988410949707),
 ('처가', 0.8120986223220825),
 ('시어머니', 0.8118054866790771),
 ('시누이', 0.8084195852279663),
 ('대소사', 0.7974411249160767),
 ('장모님', 0.7945244908332825),
 ('친정어머니', 0.7865861058235168)]

In [51]:
model_noun.wv.similar_by_word("놀이")

[('놀잇감', 0.7685993909835815),
 ('소꿉', 0.7309971451759338),
 ('공놀이', 0.7293965816497803),
 ('음률', 0.7106226682662964),
 ('누가누', 0.7102930545806885),
 ('무너뜨리기', 0.6975580453872681),
 ('장난감', 0.6944186687469482),
 ('샌드백', 0.694351315498352),
 ('보물찾기', 0.6935852766036987),
 ('칼싸움', 0.6899218559265137)]

In [52]:
model_noun.wv.similar_by_word("아이")

[('어린아이', 0.7762201428413391),
 ('지레짐작', 0.7595911026000977),
 ('엄마', 0.7447984218597412),
 ('생각', 0.7420406341552734),
 ('민영이', 0.7399120330810547),
 ('말자', 0.7398545742034912),
 ('아이라', 0.7390481233596802),
 ('기우', 0.7316569685935974),
 ('걸', 0.726838231086731),
 ('게', 0.7248141765594482)]

In [53]:
model_noun.wv.similar_by_word("치아")

[('잇몸', 0.8684966564178467),
 ('영구치', 0.832735538482666),
 ('충치', 0.8268555402755737),
 ('젖니', 0.8126301765441895),
 ('앞니', 0.789686381816864),
 ('법랑질', 0.7774391174316406),
 ('어금니', 0.7576088309288025),
 ('에나멜질', 0.7531296014785767),
 ('치석', 0.7456192970275879),
 ('치태', 0.7438891530036926)]

In [54]:
model_noun.wv.similar_by_word("이빨")

[('혓바닥', 0.7859762907028198),
 ('골짜기', 0.7575360536575317),
 ('칼자국', 0.7514342069625854),
 ('아랫입술', 0.7492984533309937),
 ('꽁지', 0.7456543445587158),
 ('거스러미', 0.7431371212005615),
 ('영인', 0.7413769960403442),
 ('혓바늘', 0.7348819971084595),
 ('생채기', 0.730977475643158),
 ('거립니다', 0.7303737998008728)]

In [55]:
model_noun.wv.similar_by_word("감기")

[('목감기', 0.8267834186553955),
 ('코감기', 0.813900887966156),
 ('환절기', 0.8049134016036987),
 ('기침감기', 0.7907943725585938),
 ('호흡기', 0.7743771076202393),
 ('편도염', 0.7707526683807373),
 ('기침', 0.7678359746932983),
 ('병치레', 0.7636606693267822),
 ('인후염', 0.7567616701126099),
 ('편도선염', 0.7561866044998169)]

In [56]:
model_noun.wv.similar_by_word("열")

[('고열로', 0.7485960125923157),
 ('오한', 0.7164791822433472),
 ('고열', 0.7153083682060242),
 ('불덩이', 0.705664336681366),
 ('외감', 0.7022899389266968),
 ('해열제', 0.6953688859939575),
 ('사병', 0.6939931511878967),
 ('熱', 0.6931891441345215),
 ('몸살감기', 0.6790531873703003),
 ('허열', 0.6735728979110718)]

In [57]:
model_noun.wv.similar_by_word("아파")

[('거립니다', 0.8615298271179199),
 ('어린애', 0.8331273794174194),
 ('참기', 0.8291198015213013),
 ('청진', 0.8153181076049805),
 ('생채기', 0.810594916343689),
 ('피멍', 0.8039909601211548),
 ('코맹맹이', 0.8034416437149048),
 ('불덩이', 0.8015407919883728),
 ('기진맥진', 0.8011857271194458),
 ('열꽃', 0.7990467548370361)]

In [58]:
model_noun.wv.similar_by_word("울음")

[('소리', 0.7191045880317688),
 ('비명', 0.7148318886756897),
 ('지르기', 0.6986531019210815),
 ('놀람', 0.6969507932662964),
 ('아달라', 0.6939250826835632),
 ('괴성', 0.6858559846878052),
 ('옹알이', 0.6853967308998108),
 ('산통', 0.6807287931442261),
 ('신음', 0.6764953136444092),
 ('큰소리', 0.6750518083572388)]

In [59]:
model_noun.wv.similar_by_word("남아", topn=20)

[('여아', 0.8833080530166626),
 ('변성기', 0.7191966772079468),
 ('기라', 0.6869825124740601),
 ('액모', 0.6843598484992981),
 ('클리토리스', 0.6725565195083618),
 ('생시', 0.6721494197845459),
 ('남근기', 0.6659187078475952),
 ('성징', 0.6620628237724304),
 ('소음경', 0.6593575477600098),
 ('대음순', 0.6578354239463806),
 ('음모', 0.6572846174240112),
 ('아도', 0.6563864350318909),
 ('인데', 0.65633624792099),
 ('음경', 0.6560852527618408),
 ('정상아', 0.6557666659355164),
 ('정류', 0.6516387462615967),
 ('색맹', 0.6507833003997803),
 ('조숙', 0.6493446826934814),
 ('여자', 0.6471385359764099),
 ('남자', 0.6467181444168091)]

In [60]:
model_noun.wv.similar_by_word("여아", topn=20)

[('남아', 0.8833080530166626),
 ('여자', 0.6963735818862915),
 ('남자', 0.6630713939666748),
 ('변성기', 0.6530897617340088),
 ('액모', 0.6377612352371216),
 ('남근기', 0.6314695477485657),
 ('성별', 0.6306352615356445),
 ('클리토리스', 0.6265316605567932),
 ('생시', 0.6179620027542114),
 ('기라', 0.614755392074585),
 ('음모', 0.6121948957443237),
 ('김우주', 0.6091545224189758),
 ('임소', 0.5993976593017578),
 ('음경', 0.5979180335998535),
 ('급성장', 0.5958536267280579),
 ('항문기', 0.5945072770118713),
 ('이지훈', 0.5915706157684326),
 ('가슴둘레', 0.590330183506012),
 ('몽우리', 0.589786171913147),
 ('페르모', 0.5878791809082031)]

In [61]:
model_noun.wv.similar_by_word("여자", topn=20)

[('남자', 0.9331412315368652),
 ('조신', 0.7207347750663757),
 ('여아', 0.6963735818862915),
 ('엘렉트라', 0.6699246168136597),
 ('여탕', 0.6697627305984497),
 ('금성', 0.6680851578712463),
 ('성차', 0.6564136743545532),
 ('남탕', 0.6510405540466309),
 ('남아', 0.6471385359764099),
 ('족쇄', 0.6469501256942749),
 ('성별', 0.6338241100311279),
 ('씩', 0.6299768686294556),
 ('아이든', 0.6263928413391113),
 ('태사혜', 0.6242064237594604),
 ('머슴', 0.6178969144821167),
 ('알파걸', 0.6169254779815674),
 ('남근', 0.6162750720977783),
 ('남학생', 0.6151830554008484),
 ('골목대장', 0.6093780994415283),
 ('야동', 0.6071764230728149)]

In [62]:
model_noun.wv.similar_by_word("남자", topn=20)

[('여자', 0.9331412315368652),
 ('조신', 0.7398948669433594),
 ('금성', 0.7089701890945435),
 ('여탕', 0.6947125196456909),
 ('성차', 0.6851638555526733),
 ('엘렉트라', 0.6845148801803589),
 ('족쇄', 0.6840630173683167),
 ('남탕', 0.6804056167602539),
 ('얌전', 0.6647123694419861),
 ('여아', 0.6630713939666748),
 ('알파걸', 0.6601797938346863),
 ('위엄', 0.6510611772537231),
 ('씩', 0.6497551202774048),
 ('남아', 0.6467181444168091),
 ('머슴', 0.6429688930511475),
 ('남근', 0.6427997946739197),
 ('성별', 0.6413998007774353),
 ('오이디푸스', 0.6394500732421875),
 ('골목대장', 0.6356361508369446),
 ('유다이', 0.6341624855995178)]

In [63]:
model_noun.wv.similar_by_word("유모차")

[('디럭스', 0.8127257823944092),
 ('절충', 0.7845213413238525),
 ('타보', 0.7658920288085938),
 ('휴대용', 0.748757004737854),
 ('핸들링', 0.736257791519165),
 ('잉글', 0.729978084564209),
 ('카시트', 0.7209837436676025),
 ('퀴니', 0.7162725925445557),
 ('페도라', 0.7005229592323303),
 ('레시', 0.7004623413085938)]

In [64]:
model_noun.wv.similar_by_word("카시트")

[('브라이', 0.7512459754943848),
 ('장착', 0.7396599054336548),
 ('안전벨트', 0.7276692986488342),
 ('유모차', 0.7209837436676025),
 ('점식', 0.705068826675415),
 ('부스터', 0.7004005908966064),
 ('순성', 0.6971040964126587),
 ('타카타', 0.6939116716384888),
 ('뒷좌석', 0.6919516921043396),
 ('컨버터블', 0.6908872127532959)]

### Volume of Corpus 

In [65]:
df[['text']].to_csv('data/text_for_word2vec.csv', index=False, encoding='utf-8')

In [66]:
pd.read_csv('data/text_for_word2vec.csv')

,text
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...
...,...
25290,임신을 축하드립니다 임신을 확인한 후에 느끼는 기쁨과 설레임 이제는 혼자가 아니기에...
25291,임신을 한 순간부터 태교는 시작된다 태교는 단순히 영재를 출산하기 위해서가 아니라 ...
25292,16 20주 6개월 태아는 엄마 배꼽 아래에 있으며 자궁은 태아가 자유롭게 움직일 ...
25293,현대의학의 힘으로는 풀아하지 못하는 영역 중의 하나가 태몽이다 신기하게 임신만 하면...


### Loading check

In [67]:
model = Word2Vec.load('data/tokmom_word2vec_only_noun.model')
model

## Doc2Vec

In [70]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [71]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['noun'])]

In [72]:
%%time
doc2vec_model = Doc2Vec(documents, 
                        vector_size=100, 
                        window=5, 
                        min_count=5, 
                        workers=4)

model.save('data/tokmom_doc2vec_only_noun.model')

CPU times: user 1min 5s, sys: 1.3 s, total: 1min 6s
Wall time: 28.1 s


In [73]:
len(doc2vec_model.docvecs)

25295

In [74]:
df['text'][0]

'6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다'

In [75]:
print("Input Document", df['text'][100], sep="\n")

similar_docs = doc2vec_model.docvecs.most_similar(100)

print("Most similar Documents")
for idx, similarity in similar_docs[:3]:
    print("idx:", idx, "similarity:", np.round(similarity, 3))
    print(df['text'][idx])

Input Document
젖병을 떼야 하는 이유로는 첫째 우식증을 예방하고 둘째 젖병 무는 어린 아기의 습관을 서서히 고쳐 나가기 위함 입니다 지금은 단지 습관에 의해 젖병을 물고 있을 뿐이므로 음식은 숟가락으로 먹이고 우유는 컵으로 마시게 하면서 젖병을 뗄 수 있도록 해야 합니다 잠잘 때 젖병을 물고 자는 것은 무의식적인 행동입니다 그러나 너무 친숙해진 습관을 갑자기 금지시킨다면 아기 심리에 큰 충격을 줄 수도 있으므로 서서히 젖병 속의 우유를 희석시키는 방법을 사용합니다 예를 들어 평상시에 우유를 200 mL 먹였다면 우유 180 mL에 물 20 mL를 섞어서 젖병에 넣어줍니다 점차 물의 농도를 높여가다가 아기가 이를 느끼고 부정적인 반응을 보일 때는 잠시 정지하였다가 다시 희석의 강도를 높여봅니다 그러다 보면 아기는 점차 희석된 우유에 흥미를 잃어 스스로 젖병을 내던지게 될 것입니다 아기에 따라 스스로 정서적 안정을 얻는 습관을 하나씩 가지고 있습니다 아기 때의 이불을 늘 잡고 있다거나 엄마의 머리카락을 잡아야 잠을 잔다거나 하는 습관은 고치기가 쉽지 않은 일입니다 젖병도 그런 습관의 일종일 수 있습니다 젖병을 물고 있는 것이 정서적인 이유라면 젖병 떼기를 강요하지 않습니다 하루 2 3번은 젖병을 물고 있게 하다가 점차 아기가 성장하면서 젖병 무는 것이 자신에게는 어울리지 않는다는 것을 엄마가 끊임없이 설득하고 아이 스스로도 그렇게 느껴 자연스레 젖병을 멀리하게 합니다 가능한 컵이나 빨대컵을 활용하고 노리개 젖꼭지 등을 대체하여 사용해 볼 수도 있습니다
Most similar Documents
idx: 14914 similarity: 0.608
젖병은 돌 전후 떼는 것이 일반적이며 늦더라도 18개월쯤에는 끊어야 한다 돌이 지나면서부터는 수유가 아닌 균형 잡힌 식사로 고르게 영양을 섭취해야 하지만 돌 무렵부터 아이는 자기주장이 생겨 고집을 피우고 싫은 건 완강하게 거부해 젖병 떼기가 더욱 힘들어진다 게다가 소근육이 발달해 직접 젖병을 들고 먹으려 하는데 

In [76]:
df['tokenized_doc'] = df['noun'].apply(lambda x: doc2vec_model.infer_vector(x).astype(float))

In [77]:
df

,text,tokenized,noun,tokenized_doc
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...,"[6, 살, 남, 아, 입, 니, 다, 회사, 복직, 후, 3, 살, 부터, 주, ...","[살, 회사, 복직, 후, 살, 주, 중, 외할머니, 댁, 살, 살, 터울, 사촌,...","[0.21359071135520935, 0.2991756200790405, -0.0..."
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...,"[요즘, 들, 어, 아빠, 가, 아이, 에게, 다가가, 려고, 하, 면, 엄마, 한...","[아빠, 아이, 엄마, 거, 엄마, 전, 정도, 엄마, 아빠, 거부, 엄마, 눈, ...","[0.0054815225303173065, 0.23851287364959717, -..."
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...,"[둘째, 가, 태어나, 면, 첫째, 가, 힘들, 거, 라는, 건, 예상, 하, 고,...","[둘째, 첫째, 거, 건, 예상, 때, 엄마, 껌, 딱지, 첫째, 외출, 때, 제외...","[0.2075052708387375, 0.34153544902801514, -0.2..."
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...,"[20, 개월, 남아, 가, 있, 고, 임신, 8, 개월, 차, 로, 두, 아기, ...","[개월, 남아, 임신, 개월, 차, 아기, 엄마, 아이, 잠자리, 독립, 생후, 일...","[-0.1482308804988861, 0.46494120359420776, -0...."
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...,"[14, 개월, 남자, 아기, 인데, 심하, 게, 깨무, 는, 버릇, 이, 있, 어...","[개월, 남자, 아기, 버릇, 전, 때, 이제, 자기, 지대, 때, 떼, 상대방, ...","[0.10979384183883667, 0.09476017951965332, -0...."
...,...,...,...,...
25290,임신을 축하드립니다 임신을 확인한 후에 느끼는 기쁨과 설레임 이제는 혼자가 아니기에...,"[임신, 을, 축하, 드립니다, 임신, 을, 확인, 한, 후, 에, 느끼, 는, 기...","[임신, 축하, 임신, 확인, 후, 기쁨, 이제, 혼자, 행동, 하나하나, 조심, ...","[0.20940089225769043, -0.2669221758842468, -0...."
25291,임신을 한 순간부터 태교는 시작된다 태교는 단순히 영재를 출산하기 위해서가 아니라 ...,"[임신, 을, 한, 순간, 부터, 태교, 는, 시작, 된다, 태교, 는, 단순히, ...","[임신, 순간, 태교, 시작, 태교, 영재, 출산, 임신, 기간, 중, 엄마, 태아...","[0.030561812222003937, -0.10744994878768921, -..."
25292,16 20주 6개월 태아는 엄마 배꼽 아래에 있으며 자궁은 태아가 자유롭게 움직일 ...,"[16, 20, 주, 6, 개월, 태아, 는, 엄마, 배꼽, 아래, 에, 있, 으며...","[주, 개월, 태아, 엄마, 배꼽, 아래, 자궁, 태아, 자유, 정도, 공간, 임신...","[0.1937972456216812, 0.046999555081129074, -0...."
25293,현대의학의 힘으로는 풀아하지 못하는 영역 중의 하나가 태몽이다 신기하게 임신만 하면...,"[현대, 의학, 의, 힘, 으로, 는, 풀, 아, 하, 지, 못하, 는, 영역, 중...","[현대, 의학, 힘, 영역, 중, 하나, 태몽, 임신, 태몽, 태몽, 아들, 딸, ...","[0.1894202083349228, 0.40511733293533325, -0.9..."
